In [6]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 

import gensim
model = gensim.models.Word2Vec.load("latest-ja-word2vec-gensim-model/word2vec.gensim.model")

import pickle
import collections

import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials

In [7]:
# 関係ベクトルをインポート

f = open("9_relatoinvec_list",'rb')
mean_vec_list = pickle.load(f)

In [8]:
# 登録済固有名詞リスト

f = open("./koyuu_noun_list.binaryfile",'rb')
noun_list = pickle.load(f)

# 関数化する
- 入力=リスト, 正解単語
- 出力=辞書

In [9]:
def output_function(imput_word_list):
    # 関係ベクトルに入力単語のベクトルを加算, 減算

    result_vec_list  = []

    for w in imput_word_list:

        w_to_v = model.wv[w]

        for mean_vec in mean_vec_list:
            result_vec_plus = w_to_v - mean_vec 
            result_vec_minus =   w_to_v + mean_vec 
    
            result_vec_list.append(result_vec_plus)
            result_vec_list.append(result_vec_minus)
    
    # 推定ベクトルと類似度が高い単語をn個上位表示
    output_word_list = []
    max_n = 10

    for vec in result_vec_list:
        most_similar = np.array(list(model.wv.most_similar([vec], [], max_n)))
        output_word_list.extend(most_similar[:,0].tolist())
    
    
    #インプットした単語と同じ単語は除去

    for imput_word in imput_word_list:
        output_word_list = [output_word for output_word in output_word_list if output_word != imput_word]
    
    # 固有名詞リスト外の単語は除去
    # これをすると単語が消える=>普通名詞の関係ベクトルは普通名詞を推定しやすい
    output_word_list = [output_word for output_word in output_word_list if noun_list.count(output_word)>=1]
    
    c = collections.Counter(output_word_list)
    c = sorted(c.items(), key=lambda x:x[1])
    
    return c

In [10]:
# 評価関数
def evaluation(c, answer):
    output_dic = {}
    
    for taple in c:
        word = taple[0]
        num = taple[1]
        
        if answer == word:
            return 1
        else:
            print("なし")
            return 0

In [11]:
# (1) Google Spread Sheetsにアクセス
def connect_gspread(jsonf,key):
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    SPREADSHEET_KEY = key
    worksheet = gc.open_by_key(SPREADSHEET_KEY).sheet1
    return worksheet

# 実行コード

In [12]:
# ここでjsonfile名と2-2で用意したkeyを入力
jsonf = "quickstart-1595337771922-22c2ceb47d3c.json"
spread_sheet_key = "1WKpQ4sxL_IP9Fm6551xDRfhxhDLzmgGAG3FCzDFmamI"
worksheet = connect_gspread(jsonf,spread_sheet_key)

In [13]:
# 入力単語とその回答をセット

for i in range(1, 41):
    row_list = worksheet.row_values(i)
    answer = row_list[0]
    imput_word_list = row_list[1:]
    
    c = output_function(imput_word_list)
    #print(c)
    evaluation(c, answer)

[('北海道電力', 1), ('バスロケーションシステム', 1), ('東電', 1), ('航空局', 1), ('日本銀行', 1), ('シアタークリエ', 1), ('コピーライターズクラブ', 1), ('アスコリ・ピチェーノ', 1), ('虎男', 1), ('ヴァカンソレイユ・', 1), ('四国', 1), ('道外', 1), ('サッター', 1), ('永代', 1), ('小繋', 1), ('アドベ', 1), ('関西電力', 2), ('環境庁', 2), ('防衛施設庁', 2), ('夏秋', 2), ('飯田橋', 2), ('西東京', 2), ('全米', 2), ('九州', 2), ('屋敷', 2), ('小作', 2), ('町', 2), ('尾口', 3), ('水産庁', 5), ('現代書館', 5), ('日教組', 5), ('ニューカマーズ', 5), ('近畿運輸局', 8), ('五月', 8), ('ロボコン', 10), ('ブラックヒルズ', 11), ('農水省', 13), ('気象庁', 17), ('東海', 19), ('六月', 26), ('社会保険庁', 28), ('富', 34), ('日本道路公団', 35), ('環境省', 44), ('税', 59), ('東日本', 62), ('林野庁', 80), ('パベルロマンメモリアル', 97), ('北海道', 108), ('関東', 108), ('東京電力', 113), ('ワイナリー', 141), ('ショットガンハウス', 142), ('田畑', 302), ('サッフォー', 400), ('澹', 400), ('運輸省', 444), ('西日本', 532), ('関西', 626), ('プランテーション', 1419), ('建設省', 1527), ('農林水産省', 1673)]
なし
